In [1]:
import scipy.stats as ss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
to_fit = pd.read_csv("Data/costs_input_0601.csv")

In [3]:
to_fit.head()

,Variable Name,Value,Lower Bound,Upper Bound
0,cInitialNICUTests,160.86,84.26,237.46
1,cSepsisNICUTests,654.36,342.76,965.96
2,cBloodCulture,37.80,19.80,55.80
3,cABX,48.99,25.66,72.32
4,cLevel1Room,420.28,294.94,545.62


In [4]:
to_fit.tail()

,Variable Name,Value,Lower Bound,Upper Bound
4,cLevel1Room,420.28,294.94,545.62
5,cLevel2Room,782.15,548.88,1015.41
6,cHealthy,221822.05,199639.85,244004.26
7,cNCD,1475666.09,1164233.60,1746350.40
8,cDeath,10920524.54,6240299.74,15600750.48


In [5]:
np.arange(65, 150, 5)

array([ 65,  70,  75,  80,  85,  90,  95, 100, 105, 110, 115, 120, 125,
       130, 135, 140, 145])

In [6]:
def fit_within_range(value, lower_bound, upper_bound, search_lower, search_upper, step_size):
    
    search_range = np.arange(search_lower, search_upper, step_size)[::-1]

    for a in search_range:
        ppf_975 = ss.gamma.ppf(0.975, a=a, scale=value / a)
        ppf_025 = ss.gamma.ppf(0.025, a=a, scale=value / a)
        
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            scale = value / a
            print("a: %.2f \t scale: %.2f" % (a, scale))           
            return a, scale
        
    return 0, 0

In [7]:
# fits an encompassing gamma distribution to an expected value of value,
# 2.5th percentile of lower_bound, and 97.5th percentile of upper_bound
# scale_lower_bound and scale_upper_bound are guesses as to what the
# scale should be.

def fit_encompassing_gamma_distr(value, lower_bound, upper_bound):
    a, b = fit_within_range(value, lower_bound, upper_bound, 350, 370, 0.1)
    if a!=0 and b!=0:
        return a, b
    
    a, b = fit_within_range(value, lower_bound, upper_bound, 75, 80, 0.1)
    if a!=0 and b!=0:
        return a, b
        
    a, b = fit_within_range(value, lower_bound, upper_bound, 4, 75, 0.1)
    if a!=0 and b!=0:
        return a, b
        
    print("No match found")
    return 0, 0

In [8]:
to_fit["a"], to_fit["scale"] = np.vectorize(fit_encompassing_gamma_distr)(to_fit["Value"], to_fit["Lower Bound"], to_fit["Upper Bound"])

a: 12.40 	 scale: 12.97
a: 12.40 	 scale: 12.97
a: 12.40 	 scale: 52.77
a: 12.40 	 scale: 3.05
a: 12.40 	 scale: 3.95
a: 36.30 	 scale: 11.58
a: 36.30 	 scale: 21.55
a: 364.70 	 scale: 608.23
a: 76.80 	 scale: 19214.40
a: 15.90 	 scale: 686825.44


In [9]:
to_fit["gamma_mean"] = to_fit["a"] * to_fit["scale"]
to_fit["gamma_2.5pct"] = ss.gamma.ppf(0.025, a=to_fit["a"], scale=to_fit["scale"])
to_fit["gamma_97.5pct"] = ss.gamma.ppf(0.975, a=to_fit["a"], scale=to_fit["scale"])

In [10]:
to_fit

,Variable Name,Value,Lower Bound,Upper Bound,a,scale,gamma_mean,gamma_2.5pct,gamma_97.5pct
0,cInitialNICUTests,160.86,84.26,237.46,12.4,12.972581,160.86,8.416316e+01,2.619842e+02
1,cSepsisNICUTests,654.36,342.76,965.96,12.4,52.770968,654.36,3.423661e+02,1.065722e+03
2,cBloodCulture,37.80,19.80,55.80,12.4,3.048387,37.80,1.977724e+01,6.156287e+01
3,cABX,48.99,25.66,72.32,12.4,3.950806,48.99,2.563194e+01,7.978744e+01
4,cLevel1Room,420.28,294.94,545.62,36.3,11.577961,420.28,2.948323e+02,5.676160e+02
5,cLevel2Room,782.15,548.88,1015.41,36.3,21.546832,782.15,5.486892e+02,1.056345e+03
6,cHealthy,221822.05,199639.85,244004.26,364.7,608.231560,221822.05,1.996376e+05,2.451585e+05
7,cNCD,1475666.09,1164233.60,1746350.40,76.8,19214.402214,1475666.09,1.164192e+06,1.823504e+06
8,cDeath,10920524.54,6240299.74,15600750.48,15.9,686825.442767,10920524.54,6.229522e+06,1.690657e+07


In [11]:
to_fit.to_csv("Data/costs_output_0601.csv")